In [ ]:

import numpy as np
from matplotlib import pyplot as plt
from scipy import stats, signal
from keras.models import Model, Sequential
from keras.layers import Conv2D,Conv1D, Dropout, MaxPooling2D, Flatten, Dense, Input , concatenate, MaxPooling1D, BatchNormalization, AveragePooling2D,AveragePooling1D
from keras.losses import categorical_crossentropy
from keras.utils import plot_model
from keras.optimizers import Adam
from keras import metrics
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer , StandardScaler

In [ ]:
drive.mount('/content/gdrive')
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Yariv
feat_space = './drive/MyDrive/audio_colab/Features/'

# Tal
#feat_space = './drive/MyDrive/Voice Recording Database/Features/'

X = {}
Y = {}

class_no = 0
stat_names =[
        'spectral_centroid',
        'spectral_bandwidth',
        'rms',
        'zero_crossing_rate']

mel_orders = ['mfcc', 'del-mfcc', 'del-del-mfcc']

for spect_path in os.listdir(feat_space):
    data = {}
    if '.npy' in spect_path and 'Copy' not in spect_path:
      label, ind = spect_path.split('_')

      obj = np.load(feat_space+spect_path, allow_pickle=True)
      stats = obj[0,0]
      data['STFT'] = obj[0,1]
      mel = obj[0,2]  
      data['Spect'] = obj[0,3]
      data['dSdT'] = obj[0,4]
      data['dS2dT2'] = obj[0,5]

      for i in range(len(stat_names)):
          data[stat_names[i]] = stats[i,:]

      for i in range(len(mel_orders)):
          data[mel_orders[i]] = mel[i, :, :]

      if label not in Y.keys():
          Y[label] = class_no
          X[class_no] = []
          class_no += 1

      this_class = Y[label]
      X[this_class].append(data)







In [ ]:
def get_model_block(inp_shape, dims, p=0.5):

    if dims=='image':

      inp = Input(shape= (inp_shape[0], inp_shape[1], 1))
      x = Conv2D(256, kernel_size=(3,3), activation='relu', input_shape=inp_shape)(inp)
      x = AveragePooling2D(pool_size=(2,2))(x)
      x = Dropout(p)(x)
      x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
      x = AveragePooling2D(pool_size=(2,2))(x)
      x = Dropout(p)(x)
      x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
      x = AveragePooling2D(pool_size=(2,2))(x)
      x = Dropout(2*p)(x)
      x = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
      x = AveragePooling2D(pool_size=(2,2))(x)
      x = Dropout(3*p)(x)
      
  
      connection_layer = Flatten()(x)

    if dims=='coeffs':

      inp = Input(shape= (inp_shape[0], inp_shape[1], 1))
      x = Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=inp_shape)(inp)
      x = MaxPooling2D(pool_size=(2,2))(x)
      x = Dropout(p)(x)
      x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
      x = MaxPooling2D(pool_size=(2,2))(x)
      x = Dropout(2*p)(x)
      x = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
      x = MaxPooling2D(pool_size=(2,2))(x)
      x = Dropout(3*p)(x)
      
      connection_layer = Flatten()(x)

    if dims=='stats':
      inp = Input(shape = (inp_shape[0],1) )
      x = Conv1D(128, kernel_size=3, activation='relu', input_shape=inp_shape)(inp)
      x = MaxPooling1D(pool_size=2)(x)
      x = Dropout(p)(x)
      x = Conv1D(64, kernel_size=3, activation='relu')(x)
      x = MaxPooling1D(pool_size=2)(x)
      x = Dropout(2*p)(x)
      x = Conv1D(32, kernel_size=3, activation='relu')(x)
      x = MaxPooling1D(pool_size=2)(x)
      x = Dropout(3*p)(x)

      connection_layer = Flatten()(x)

    m = Model(inputs = inp,outputs = connection_layer)

    return m

def model_output(models,num_class,lr=1e-3, p=0.5):

    if len(models) > 1 :
      outputs = concatenate([ m.output for m in models ])
      inputs = [ m.input for m in models ] 

    else:
      outputs = concatenate([models[0].output,models[0].output])
      inputs = [models[0].input]

    x = Dense(32, activation='relu')(outputs)
    x = Dropout(p)(x)
    output_layer = Dense(num_class, activation='softmax')(outputs)

    opt = Adam(lr)
    m = Model(inputs = inputs ,outputs = output_layer)
    m.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    return m 

In [ ]:
def normalize_reshape(x_train, x_test, feats):
  Xtrain = {}
  Xtest = {}

  for f in feats:
    train_len = len(x_train)
    test_len = len(x_test)
    orig_shape = x_train[0][f].shape
    dims = len(orig_shape)
    norm = Normalizer()

    if dims > 1:
      h,w = orig_shape
      X0 = np.array([x[f].flatten() for x in x_train])
      Xf = np.array([x[f].flatten() for x in x_test])

      X0 = norm.fit_transform(X0).reshape(train_len, h, w)
      Xf = norm.transform(Xf).reshape(test_len, h, w)

    else:
      l = orig_shape[0]
      X0 = np.array([x[f] for x in x_train])
      Xf = np.array([x[f] for x in x_test])

      X0 = norm.fit_transform(X0).reshape(train_len, l, 1)
      Xf = norm.transform(Xf).reshape(test_len, l, 1)

    Xtrain[f] = X0
    Xtest[f] = Xf

  return Xtrain, Xtest

In [ ]:
def wrapper(_X_train,_Y_train2,dim,feats):

  done=False
  best_loss = 100
  best_feats = []
  counter = 0
  lr=1e-3
  batch_size=16
  dropout_block= 0.2
  dropout_out= 0.7
  epochs=85
  split = 0.7
  v = 0

  while not done:
    losses = []
    x_test_round, x_train_round, y_test_round, y_train_round = train_test_split(_X_train, _Y_train2, test_size=split, random_state=7)
    x_train_round, x_test_round = normalize_reshape(x_train_round, x_test_round, feats)
    m_list = []
    x_train = []
    x_test = []

    if len(best_feats):
      
      for f in best_feats:
        
        x_train.append(x_train_round[f])
        x_test.append(x_test_round[f])

        m_list.append(get_model_block(x_train_round[f][0].shape, dims[f], p=dropout_block))
      #print('best features: ',best_feats)
      #print('best loss :',best_loss)

    for f in feats:
      if f not in best_feats:
        #print('add '+f)
        x_train_i = []
        x_test_i = []

        x_train_i.extend([block for block in x_train])
        x_test_i.extend([block for block in x_test])

        x_train_i.append(x_train_round[f])
        x_test_i.append(x_test_round[f])

        m_list.append(get_model_block(x_train_round[f][0].shape,dims[f], p=dropout_block))

        m = model_output(m_list, num_class=Y_train.max()+1,lr=lr, p=dropout_out)

        if len(m_list) > 1:
          h = m.fit(x_train_i, y_train_round, epochs=epochs, batch_size=batch_size, verbose=v)
          loss, acc = m.evaluate(x_test_i, y_test_round, batch_size=batch_size)
        else:
          h = m.fit(x_train_i, y_train_round, epochs=epochs, batch_size=batch_size, verbose=v)
          loss, acc = m.evaluate(x_test_i, y_test_round, batch_size=batch_size)

        #print('train acc:' + str(max(h.history['accuracy'])))

        losses.append(loss)
        m_list.pop()

    best_round = min(losses)
    best_feat_round = losses.index(best_round) 
    
    if best_round < best_loss*0.99:
      best_loss = best_round
      best_feats.append(feats[best_feat_round+counter])
      hyper_params = [lr,batch_size,dropout_block,dropout_out,epochs,split] 
      counter+=1
      
    else:
      break

    if len(best_feats) == 3:
      print('best feats: ', best_feats)
      print(hyper_params)
      break

    else:
      r=np.random.uniform(0.0001,0.001)
      batch_size= np.random.randint(16,28)
      dropout_block= np.random.uniform(0.1,0.3)
      dropout_out= np.random.uniform(0.5,0.7)
      epochs=np.random.randint(40,80)
      split = np.random.uniform(0.7,0.75)
      
  return sorted(best_feats, key=str.lower), hyper_params, best_loss

In [ ]:
def extract_best_feat(_best_feats,best_params,best_loss):

  freq_dict = {}
  losses_best_feat = []
  min_key_loss = 100 

  for feat in _best_feats :
    feat = tuple(feat)

    if feat not in freq_dict.keys():
      freq_dict[feat] = 1
    else:
      freq_dict[feat] += 1 

  keys = list(freq_dict.keys()) 
  keys = [ list(key) for key in keys]
  vals = freq_dict.values()
  vals = [ val for val in vals ]
  
  max_counter_idx = np.argmax(vals)
  print('counts of same feats :',vals)
  max_key = list(keys[max_counter_idx])

  for i in range(len(_best_feats) ) :
    
    if _best_feats[i] == max_key :
      losses_best_feat.append(best_loss[i])

      if best_loss[i] < min_key_loss :
        min_key_loss = best_loss[i]
        max_key_params =hyper_params[i]

  fig, ax = plt.subplots()
  fig.set_figheight(10)
  fig.set_figwidth(40)
  ax.bar(range(len(vals) ),vals)
  ax.set_title('features histogram')
  ax.set_xticks(range(len(vals) ))
  ax.set_xticklabels(keys,fontsize = 'xx-small')
  plt.show(ax)

  plt.figure(figsize=(8,8))
  hist_losses_best_feat = plt.hist(losses_best_feat,bins = round(len(losses_best_feat)/2 ) )
  plt.title('loss histogram')
  plt.show(hist_losses_best_feat)

  plt.figure(figsize=(8,8))
  hist_losses = plt.hist(best_loss,bins = round(len(best_loss)/4 ) )
  plt.title('all loss histogram')
  plt.show(hist_losses)

  return max_key ,min_key_loss , max_key_params


In [ ]:
X_train = []
Y_train = []
for i in range(class_no):
    X_train.extend(X[i])
    Y_train.extend([i for l in range(len(X[i]))])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
feats = ['mfcc', 'del-mfcc','del-del-mfcc', 'Spect', 'rms', 'dSdT', 'dS2dT2','zero_crossing_rate', 'spectral_centroid', 'spectral_bandwidth']
dims = {
        'STFT':'image',
        'Spect':'image',
        'dSdT':'image',
        'dS2dT2':'image',
        'mfcc':'coeffs',
        'del-mfcc':'coeffs',
        'del-del-mfcc':'coeffs',
        'rms':'stats',
        'zero_crossing_rate':'stats',
        'spectral_centroid':'stats',
        'spectral_bandwidth':'stats'
        }

classes = Y_train.max()+1
class_labels = np.zeros((classes))
Y_train2 = []

for y in Y_train:
    t = np.copy(class_labels)
    t[y] = 1
    Y_train2.append(t)

Y_train2 = np.array(Y_train2)

In [ ]:
# finding best of the best
rounds = 100
best_feats = []
hyper_params = []
best_loss = []

for i in range(rounds):

  best_feats_round, hyper_params_round, best_loss_round = wrapper(X_train,Y_train2,dims,feats)

  best_feats.append(best_feats_round)
  hyper_params.append(hyper_params_round)
  best_loss.append(best_loss_round)

  print(best_feats[i])
  print(hyper_params[i])
  print(best_loss[i]) 
  print('round:'+str(i) )




In [ ]:
best_feats = np.array(best_feats)
hyper_params = np.array(hyper_params)
best_loss = np.array(best_loss)
np.save(feat_space+'best_feats',best_feats )
np.save(feat_space+'hyper_params',hyper_params )
np.save(feat_space+'best_loss',best_loss )



In [ ]:
best_feats = np.load(feat_space+'best_feats.npy', allow_pickle=True)
hyper_params = np.load(feat_space+'hyper_params.npy', allow_pickle=True)
best_loss = np.load(feat_space+'best_loss.npy', allow_pickle=True)

In [ ]:
max_key, min_key_loss, max_key_params =  extract_best_feat(best_feats,hyper_params,best_loss)
print('most frequent best features: ', max_key)
print('min loss for best features: ',min_key_loss)
print('params for best features: ',max_key_params)

In [ ]:
plot_model(m,show_shapes=True)